# Traitement des données pour un fichier

Le but de ce fichier est de présenter le pré-traitement des données, avant l'utilisation d'un module d'IA.<br>
Pour réliser cela il faut :
- réduire au maximum le nombre de points pour permettre un apprentissage efficace même sur peu de données
- conserver toutes les informations permettant de différencier une courbe normale d'une courbe de malade


Import des modules utiles :
- `pandas` pour le traitement des données 
- `numpy` pour les calcules sur les vecteurs
- `scipy.signal` pour faire du traitement du signal

Pour avoir plus d'information sur les fonction créées pour traiter les données voir [tools](tools.py).

Remarque : on utilisera `plotly` pour l'affichage des courbes, il faut donc l'avoir installé.

In [261]:
import pandas as pd
import numpy as np
from importlib import reload

import tools

pd.options.plotting.backend = "plotly"
reload(tools)

<module 'tools' from 'c:\\Users\\jleva\\Documents\\Telecom\\2A\\stage\\GCMS\\tools.py'>

Définition des constantes qui seront utiles dans la suite du programme :
- `PATH`, le chemin jusqu'au fichier csv étudié
- `PERIOD`, la periode de ré-échantillonage des données en seconde, permettant de conserver plus ou moins de points sur la courbe
- `INTERVAL`, l'intervalle de temps contenant les informations significatives en minutes
- `NOISE_FREQ`, fréquence de coupure du filtre passe-bas permettant d'éliminer le bruit en Hz, valeur de None si il n'y a pas de filtrage souhaité
- `BIAS_FREQ`, fréquence de coupure du filtre passe-bas permettant de déterminer le biais sur la courbe en Hz
- `THRESHOLD`, seuil à partir duquel on considère une valeur comme significative dans la mesure.
- `SPIKES`, temps d'apparition en minute des trois pics de référence, valeur de None si le pic n'apparait pas

In [262]:
PATH = '../donnees/export-chromato/ChedadiA-20201223-CHROMATOGRAM.CSV'
print(tools.PERIOD) 
print(tools.RESAMPLE_MS)
print(tools.INTERVAL)
print(tools.NOISE_FREQ)
print(tools.BIAS_FREQ) 
print(tools.THRESHOLD)
print(tools.SPIKES) 

[1, 2, 6, 20]
['1s', '2s', '6s', '20s']
[6.01, 45.01]
None
0.002
3.4
[22.528, 23.749, 38.614]


## Lecture d'un fichier csv et affichage des données
On utilise pandas pour lire directement le fichier csv et mettre les données dans un objet du type DataFrame.<br>
Les 5 premières lignes ne sont pas prises en compte car elles correspondent aux métadonnées de l'analyse. <br>
Le parser détecte trois colonnes car les lignes finissent par une virgule, il faut donc supprimer la troisième colonne, qui est vide, du DataFrame.

Voici un affiche des données brutes :

In [263]:
df = tools.readCSV(PATH)
df.plot()

Pour avoir une meilleur généralisation sur tous les types de courbes, il est possible de compenser les écart de temps qui peuvent être introduits par les maintenances par exemple ou les variabilités au cours du temps induite par la machine. Pour corriger cela on peut se baser sur trois pics de référence, qui sont des molécules toujours présentes sur le graphique (attention le deuxième pic peut ne pas être présent en cas de prise de certains médicaments par exemple). Pour corriger les variabilités en temps, on peut aligner ces pics de référence sur des repères fixes. <br>

Remarque : ici cela est pésenté pour l'exemple, la fonction `adaptCurve()` fait déjà ces traitement.


In [264]:
print('Repères : ', tools.SPIKES_EXPECTED_TIME)
print('Pics de référence détectés : ', tools.SPIKES)

Repères :  [21.5, 23, 38]
Pics de référence détectés :  [22.528, 23.749, 38.614]


In [265]:
temp = tools.alignSpikes(df, tools.SPIKES)

On ne garde que les valeurs comprises dans l'intervalle qui nous interesse, cela supprime aussi les cases éventuellement vide du csv (temps de 0 à la lecture)<br>
Voici un affichage des données sur l'intervalle voulu :

In [266]:
temp = temp.drop(temp[temp.index >= tools.INTERVAL[1]].index)
temp = temp.drop(temp[temp.index <= tools.INTERVAL[0]].index)
print(len(temp))
temp.plot()

6677


## Mise en évidence des pics
Le passage des données en logarithme permet de mettre en évidence les plus petits pics du graphique, le but est de ne pas donner un importance trop grande aux pics les plus grands, car cela pourrait compliquer le travail sur la partie IA dans le cas ou l'information importante est contenue dans un pic de taille faible.<br>
Remarque : la fonction logarithme est bijective, il n'y a donc pas de perte d'information.

In [267]:
temp = np.log(temp)
temp.plot()

Ensuite on fait une normalisation des points de la courbe

In [268]:
temp = (temp - temp.mean())/temp.std()
temp.plot()

## Intervalles de mesures constant
La machine faisant les mesures n'a pas une période d'échantillonage constante, cela est problématique pour l'exploitation des données car chaque courbes aura des points d'échantillonage différents. <br>
Plus la période d'échantionnage sera grande plus la quantité de points à analyser dans la partie IA sera faible, il faut donc trouver un équilibre pour garder un nombre de points suffissants pour que la courbe soit exploitable, tout en limitant leur nombre afin que la solution d'IA soit en mesure d'appendre éfficacement sur la base de donnée. <br> 
On utilise donc la fonction `resample` de `pandas` qui permet de donner un échantilonnage constant Associé à `max` qui permet de conserver le maximum des points qui se trouveront entre deux points d'échantilonnage, dans le but de conserver la taille des différents pics.<br>
Ensuite `interpolate` permet de faire une interpolation polynomiale permettant de completer les points qui n'ont pas encore de valeur.

Dans un chromatogramme, tous les points n'ont pas besoin de la même précision. Il faudra être aussi précis que possible sur les intervalles : [9, 10], [25, 26.5], [29, 30]. Au contraire la zone [32, 38] contiendra peu de pics donc necessite moins de précision.

Avant de faire ceci il faut commencer par convertir le temps qui est un valeur en minutes dans un format compréhensible par `pandas`.

In [269]:
newdf = tools.readCSV(PATH)
newdf = tools.adaptCurve(newdf, [22.528, 23.749, 38.614])
print('nouveau nombre de valeur : ', len(newdf), '  ancien nombre de valeur : ', len(df))
newdf.plot()

nouveau nombre de valeur :  492   ancien nombre de valeur :  9148


## Filtrage et suppression du biais
On remarque sur la courbe ci-dessus que la machine à tendance à ajouter un biais au cours du temps. Les valeurs en 0 sont proche de -1 alors qu'en 45 elle sont plutot proche de 0.4. <br>
L'idée est donc de récupérer la composante basse fréquence de la courbe correspondant à ce biais par un filtre passe-bas, et ensuite de la soustraire aux valeurs de la courbe pour le comprenser. <br>
Comme la composante basse fréquence est influencée par les pics, on utilise la fonction `rolling` qui nous permet de prendre le minimum de la courbe sur une fenêtre glissante. En appliquant un filtre passe bas sur cette courbe on obtient donc l'allure du minimum de la courbe, ce qui correpond pour nous au biais. <br>
On remarque aussi que le filtre à tendance à avoir des effets de bords (ocillations en 0), cela est du au fait que le filtre est initialisé à 0. Pour compenser on peut ajouter des valeurs au début et à la fin du tableau avant filtrage afin de limiter ces effets. <br>
Une second filtre passe-bas pourra être utilisé pour supprimer les hautes fréquences, éliminant le bruit de mesure, mais cela à tendance à changer la hauteur des pics les uns par rapport aux autres, ce qui n'est pas une bonne chose pour la suite.

Mise en place du filtre passe-bas avec `butter` de `scipy`.<br>
Tiré de : <https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units>

Dans un premier temps recherche du minimum de la courbe au cours du temps

In [270]:
bias = newdf['values'].rolling(15).min()
bias = pd.concat([pd.DataFrame([np.NaN] * tools.PADDING), bias]) # pour compenser le padding du filtre à 0 et éliminer l'effet de bord
bias = pd.concat([bias, pd.DataFrame([np.NaN] * tools.PADDING)])
bias = bias.fillna(method='ffill') # on retire les Nan en forward et backward (pour être sûr qu'il n'y en ai plus)
bias = bias.fillna(method='bfill')
newdf['bias'] = bias.to_numpy()[tools.PADDING : -tools.PADDING] # affichage uniquement de la partie qui nous intéresse
newdf.plot()

Ensuite application du filtre passe bas sur ce minimum.

In [271]:
bias = tools.butter_lowpass_filter(bias.squeeze(), tools.BIAS_FREQ, 1 / tools.PERIOD[1])
newdf['bias'] = bias[tools.PADDING : -tools.PADDING]
newdf.plot()

On observe que le biais n'est pas bien calculé, car les différents intervalles ont une periode d'échantillonnage différente.<br>
Il faut donc calculer en prenant en compte cette periode, pour chaque intervalle. <br>
C'est le but de la fonction `computeBiasByPart`.

In [272]:
bias2 = tools.computeBiasWithResample(newdf).iloc[tools.PADDING:-tools.PADDING]
newdf['bias'] = tools.resizeBias(newdf, bias2)
newdf.plot()

Filtrage des hautes fréquences et soustraction du biais :

In [273]:
if (tools.NOISE_FREQ is None):
    newdf['corrected'] = newdf['values'] - newdf['bias']
else :
    newdf['corrected'] = tools.butter_lowpass_filter(newdf['values'], tools.NOISE_FREQ, 1/tools.PERIOD[1]) # on perd la hauteur relative entre les pics parfois
    newdf['corrected'] = newdf['corrected'] - newdf['bias']
newdf.plot()

Pour éliminer les données parasites restantes, l'idée est de faire un seuillage pour ne conserver que les valeurs significatives sur la courbe et ainsi éliminer l'impact du bruit de mesure.

In [274]:
newdf.loc[newdf['corrected'] < tools.THRESHOLD, 'corrected'] = 0
print('Nombre de valeurs restantes : ', len(newdf['corrected']))
newdf['corrected'].plot()

Nombre de valeurs restantes :  492


Pour faire tous les traitements d'un coup, il y a la fonction `readAndAdaptDataFromCSV`, utilisée comme suit : 

In [277]:
reload(tools)
other = tools.readAndAdaptDataFromCSV(PATH[:27], PATH[27:])
print(other.df)
other.df.plot()

           values
6.013833      0.0
6.113833      0.0
6.213833      0.0
6.313833      0.0
6.413833      0.0
...           ...
44.506560     0.0
44.606560     0.0
44.706560     0.0
44.806560     0.0
44.906560     0.0

[492 rows x 1 columns]


Pour lire des données d'un dossier complet il y a la fonction `readAllData` qui retourne une liste des courbes, voir ce qui suit.

In [280]:
path = '../donnees/export-chromato/'
data = tools.readAllData(path)
print('Nombre de fichiers trouvés : ', len(data))
a = len(data[0].df)
b = True
for d in data:
    if(len(d.df) != a):
        b = False
print(b)
data[56].df.plot()

Nombre de fichiers trouvés :  328
False
